## 赛事背景
讯飞开放平台针对不同行业、不同场景提供相应的AI能力和解决方案，赋能开发者的产品和应用，帮助开发者通过AI解决相关实际问题，实现让产品能听会说、能看会认、能理解会思考。

用户新增预测是分析用户使用场景以及预测用户增长情况的关键步骤，有助于进行后续产品和应用的迭代升级。

## 赛事任务
本次大赛提供了讯飞开放平台海量的应用数据作为训练样本，参赛选手需要基于提供的样本构建模型，预测用户的新增情况。

## 评审规则

### 数据说明
| 字段名称             | 字段含义                                       |
| :--------------- | :----------------------------------------- |
| mid              | 用户行为模块id                                   |
| eid              | 用户行为事件id                                   |
| did              | 用户id                                       |
| device\_brand    | 设备品牌/厂商                                    |
| ntt              | 网络类型                                       |
| operator         | 运营商                                        |
| common\_country  | 国家                                         |
| common\_province | 省份                                         |
| common\_city     | 城市                                         |
| appver           | 应用版本                                       |
| channel          | 应用渠道                                       |
| common\_ts       | 事件发生时间（毫秒时间戳）                              |
| os\_type         | 用于判断Android还是iOS                           |
| udmap            | 事件自定义属性（标准json文本，内含botId助手ID和pluginId插件ID） |
| is\_new\_did     | 预测目标，即是否为新增用户                              |

### 评估指标
本次竞赛的评价标准采用f1_score，分数越高，效果越好。

In [41]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
import lightgbm as lgb
from sklearn.ensemble import HistGradientBoostingClassifier

In [32]:
train = pd.read_csv("train.csv")
test = pd.read_csv("testA_data.csv")

train["common_ts"] = pd.to_datetime(train["common_ts"], unit="ms")
test["common_ts"] = pd.to_datetime(test["common_ts"], unit="ms")

In [33]:
train["common_month"] = train["common_ts"].dt.month
test["common_month"] = test["common_ts"].dt.month

train["common_day"] = train["common_ts"].dt.day
test["common_day"] = test["common_ts"].dt.day

train["common_hour"] = train["common_ts"].dt.hour
test["common_hour"] = test["common_ts"].dt.hour

In [34]:
train.shape, test.shape

((3429925, 18), (1143309, 17))

In [35]:
train.columns

Index(['mid', 'eid', 'did', 'device_brand', 'ntt', 'operator',
       'common_country', 'common_province', 'common_city', 'appver', 'channel',
       'common_ts', 'os_type', 'udmap', 'is_new_did', 'common_month',
       'common_day', 'common_hour'],
      dtype='object')

In [36]:
train.describe(include="all")

/var/folders/dl/0x0qfc0s6vd9wdk53j4thvnh0000gn/T/ipykernel_9042/39591749.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  train.describe(include="all")


,mid,eid,did,device_brand,ntt,operator,common_country,common_province,common_city,appver,channel,common_ts,os_type,udmap,is_new_did,common_month,common_day,common_hour
count,3.429925e+06,3.429925e+06,3429925,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06,3429925,3.429925e+06,3429925,3.429925e+06,3.429925e+06,3.429925e+06,3.429925e+06
unique,NaN,NaN,270837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3254416,NaN,8077,NaN,NaN,NaN,NaN
top,NaN,NaN,20cd6a7d3a60fd193d925b21af6660f1e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-13 00:05:47.273000,NaN,{},NaN,NaN,NaN,NaN
freq,NaN,NaN,68403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41,NaN,3162776,NaN,NaN,NaN,NaN
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-02-28 16:00:00.115000,NaN,NaN,NaN,NaN,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-31 15:59:57.196000,NaN,NaN,NaN,NaN,NaN,NaN
mean,2.264608e+01,1.366922e+02,NaN,8.892087e+01,2.605180e+00,1.929892e+00,8.091360e+01,1.459701e+02,2.400957e+02,5.874532e+01,5.914274e+00,NaN,6.228171e-01,NaN,1.560340e-01,2.997912e+00,1.578363e+01,8.535301e+00
std,1.393127e+01,7.687001e+01,NaN,5.289133e+01,1.148252e+00,1.140171e+00,2.438861e+00,7.985908e+01,1.414514e+02,2.818215e+01,4.161772e+00,NaN,4.846814e-01,NaN,3.628876e-01,4.564792e-02,8.624008e+00,5.391190e+00
min,0.000000e+00,0.000000e+00,NaN,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00
25%,1.300000e+01,6.400000e+01,NaN,5.800000e+01,2.000000e+00,1.000000e+00,8.100000e+01,7.900000e+01,8.900000e+01,2.600000e+01,2.000000e+00,NaN,0.000000e+00,NaN,0.000000e+00,3.000000e+00,8.000000e+00,4.000000e+00


In [37]:
for col in ['mid', 'eid', 'did', 'device_brand', 'ntt', 'operator',
       'common_country', 'common_province', 'common_city', 'appver', 'channel',
       'os_type']:
    train[col + "_count"] = train[col].map(train[col].value_counts())
    test[col + "_count"] = test[col].map(test[col].value_counts())

    train[col + "_target"] = train[col].map(train.groupby(col)["is_new_did"].mean())
    test[col + "_target"] = test[col].map(train.groupby(col)["is_new_did"].mean())

In [42]:
pred = cross_val_predict(
    HistGradientBoostingClassifier(),
    train.drop(["did", "udmap", "is_new_did", "common_ts"], axis=1),
    train["is_new_did"]
)
f1_score(train["is_new_did"], pred)

0.9050957798130362

In [54]:
model = lgb.LGBMClassifier()
model.fit(
    train.drop(["did", "udmap", "is_new_did", "common_ts"], axis=1),
    train["is_new_did"]
)
pred = model.predict(test.drop(["did", "udmap", "common_ts"], axis=1))

[LightGBM] [Info] Number of positive: 535185, number of negative: 2894740
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.039179 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3023
[LightGBM] [Info] Number of data points in the train set: 3429925, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.156034 -> initscore=-1.688038
[LightGBM] [Info] Start training from score -1.688038


In [55]:
pd.DataFrame({"is_new_did": pred}).to_csv("submit.csv", index=None)